# Playground

Load some job's execution results

In [68]:
import pandas as pd

file_path = 'data/job_runs_3_days.csv'
date_columns = ['CREATE_TS', 'STATUS_UPDATE_TS']
job_runs = pd.read_csv(
    file_path, 
    parse_dates=date_columns
)

job_runs.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/job_runs_3_days.csv'

In [65]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the categorical column
job_runs['JOB_NAME_ENCODE'] = label_encoder.fit_transform(job_runs['JOB_NAME'])
job_runs['DURATION'] = (job_runs['STATUS_UPDATE_TS'] - job_runs['CREATE_TS'])
job_runs['DURATION_IN_SEC'] = job_runs['DURATION'].map(lambda delt: delt.total_seconds())

display(job_runs.head())
display(job_runs.dtypes)

,JOB_NAME,PROCESS_RUN_SK,BATCH_SK,BATCH_STATUS,CREATE_TS,STATUS_UPDATE_TS,ROWS_ADDED_COUNT,ROWS_UPDATED_COUNT,ROWS_DELETED_COUNT,ROWS_DUPLICATED_COUNT,ROWS_SKIPPED_COUNT,ROWS_ERROR_COUNT,TARGET_ENTITY,JOB_NAME_ENCODE,DURATION,DURATION_IN_SEC
0,PRES2K_PROD_CONFIG--GENERIC,3416062278,1075302813,COMPLETED,2024-02-15 21:05:39+00:00,2024-02-15 21:05:44+00:00,-1,-1,-1,-1,-1,-1,NaN,2,0 days 00:00:05,5.0
1,EXT_HANA_SEMS_ISSUE_TXN--GENERIC,3416061876,1075302612,COMPLETED,2024-02-15 21:02:18+00:00,2024-02-15 21:07:26+00:00,2617,-1,-1,-1,-1,-1,NaN,1,0 days 00:05:08,308.0
2,EXP_ZTASSETDISPOSE--GENERIC,3416061810,1075302580,COMPLETED,2024-02-15 21:01:46+00:00,2024-02-15 21:07:50+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:06:04,364.0
3,PRES2K_PROD_CONFIG--GENERIC,3416060456,1075301903,COMPLETED,2024-02-15 20:50:29+00:00,2024-02-15 20:50:33+00:00,-1,-1,-1,-1,-1,-1,NaN,2,0 days 00:00:04,4.0
4,EXT_HANA_SEMS_ISSUE_TXN--GENERIC,3416060062,1075301705,COMPLETED,2024-02-15 20:47:11+00:00,2024-02-15 20:52:03+00:00,3258,-1,-1,-1,-1,-1,NaN,1,0 days 00:04:52,292.0


JOB_NAME                              object
PROCESS_RUN_SK                         int64
BATCH_SK                               int64
BATCH_STATUS                          object
CREATE_TS                datetime64[ns, UTC]
STATUS_UPDATE_TS         datetime64[ns, UTC]
ROWS_ADDED_COUNT                       int64
ROWS_UPDATED_COUNT                     int64
ROWS_DELETED_COUNT                     int64
ROWS_DUPLICATED_COUNT                  int64
ROWS_SKIPPED_COUNT                     int64
ROWS_ERROR_COUNT                       int64
TARGET_ENTITY                        float64
JOB_NAME_ENCODE                        int64
DURATION                     timedelta64[ns]
DURATION_IN_SEC                      float64
dtype: object

In [34]:
feature_columns = ['JOB_NAME_ENCODE', 'DURATION_IN_SEC']
target_columns = ['ROWS_ADDED_COUNT', 'ROWS_UPDATED_COUNT', 'ROWS_DELETED_COUNT', 'ROWS_DUPLICATED_COUNT', 'ROWS_SKIPPED_COUNT', 'ROWS_ERROR_COUNT']
X_train = job_runs.loc[:, feature_columns +target_columns ]

X_train

X_train.head()

,JOB_NAME_ENCODE,DURATION_IN_SEC,ROWS_ADDED_COUNT,ROWS_UPDATED_COUNT,ROWS_DELETED_COUNT,ROWS_DUPLICATED_COUNT,ROWS_SKIPPED_COUNT,ROWS_ERROR_COUNT
0,2,5.0,-1,-1,-1,-1,-1,-1
1,1,308.0,2617,-1,-1,-1,-1,-1
2,0,364.0,55175,-1,-1,-1,-1,-1
3,2,4.0,-1,-1,-1,-1,-1,-1
4,1,292.0,3258,-1,-1,-1,-1,-1


In [35]:
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
import numpy as np


# Create and train the OCSVM model
ocsvm = OneClassSVM(kernel='rbf', nu=0.05)

ocsvm.fit(X_train)

# Predict anomalies (1 for normal instances, -1 for anomalies)
predictions = ocsvm.predict(X_train)

# Print predictions
print("Predictions:", predictions)


Predictions: [1 1 1 ... 1 1 1]


In [45]:
# Find anomalies
y_pred = pd.Series(predictions, name='pred')
y_pred[y_pred == -1]

17     -1
26     -1
47     -1
49     -1
114    -1
123    -1
146    -1
149    -1
222    -1
255    -1
273    -1
278    -1
281    -1
285    -1
304    -1
328    -1
339    -1
433    -1
436    -1
568    -1
627    -1
725    -1
854    -1
916    -1
999    -1
1002   -1
Name: pred, dtype: int64

In [48]:
job_runs['ANOMALY'] = y_pred


,JOB_NAME,PROCESS_RUN_SK,BATCH_SK,BATCH_STATUS,CREATE_TS,STATUS_UPDATE_TS,ROWS_ADDED_COUNT,ROWS_UPDATED_COUNT,ROWS_DELETED_COUNT,ROWS_DUPLICATED_COUNT,ROWS_SKIPPED_COUNT,ROWS_ERROR_COUNT,TARGET_ENTITY,JOB_NAME_ENCODE,DURATION,DURATION_IN_SEC,ANOMALY
17,EXP_ZTASSETDISPOSE--GENERIC,3416052758,1075298053,COMPLETED,2024-02-15 19:46:19+00:00,2024-02-15 19:49:05+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:02:46,166.0,-1
26,EXP_ZTASSETDISPOSE--GENERIC,3416047368,1075295358,COMPLETED,2024-02-15 19:01:24+00:00,2024-02-15 19:08:23+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:06:59,419.0,-1
47,EXP_ZTASSETDISPOSE--GENERIC,3416034726,1075289037,COMPLETED,2024-02-15 17:16:03+00:00,2024-02-15 17:18:52+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:02:49,169.0,-1
114,EXP_ZTASSETDISPOSE--GENERIC,3415995178,1075269263,COMPLETED,2024-02-15 11:46:29+00:00,2024-02-15 11:49:24+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:02:55,175.0,-1
123,EXP_ZTASSETDISPOSE--GENERIC,3415989834,1075266591,COMPLETED,2024-02-15 11:01:57+00:00,2024-02-15 11:09:19+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:07:22,442.0,-1
222,EXP_ZTASSETDISPOSE--GENERIC,3415930306,1075236828,COMPLETED,2024-02-15 02:45:54+00:00,2024-02-15 02:48:39+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:02:45,165.0,-1
255,EXP_ZTASSETDISPOSE--GENERIC,3415910656,1075227002,COMPLETED,2024-02-15 00:02:08+00:00,2024-02-15 00:10:59+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:08:51,531.0,-1
273,EXP_ZTASSETDISPOSE--GENERIC,3415899678,1075221513,COMPLETED,2024-02-14 22:30:40+00:00,2024-02-14 22:38:05+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:07:25,445.0,-1
285,EXP_ZTASSETDISPOSE--GENERIC,3415892442,1075217896,COMPLETED,2024-02-14 21:30:22+00:00,2024-02-14 21:37:08+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:06:46,406.0,-1
304,EXP_ZTASSETDISPOSE--GENERIC,3415881756,1075212552,COMPLETED,2024-02-14 20:01:18+00:00,2024-02-14 20:09:33+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:08:15,495.0,-1


In [49]:
job_runs[ (job_runs['JOB_NAME'] == 'EXP_ZTASSETDISPOSE--GENERIC')]

,JOB_NAME,PROCESS_RUN_SK,BATCH_SK,BATCH_STATUS,CREATE_TS,STATUS_UPDATE_TS,ROWS_ADDED_COUNT,ROWS_UPDATED_COUNT,ROWS_DELETED_COUNT,ROWS_DUPLICATED_COUNT,ROWS_SKIPPED_COUNT,ROWS_ERROR_COUNT,TARGET_ENTITY,JOB_NAME_ENCODE,DURATION,DURATION_IN_SEC,ANOMALY
2,EXP_ZTASSETDISPOSE--GENERIC,3416061810,1075302580,COMPLETED,2024-02-15 21:01:46+00:00,2024-02-15 21:07:50+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:06:04,364.0,1
5,EXP_ZTASSETDISPOSE--GENERIC,3416059830,1075301589,COMPLETED,2024-02-15 20:45:15+00:00,2024-02-15 20:48:47+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:03:32,212.0,1
8,EXP_ZTASSETDISPOSE--GENERIC,3416058102,1075300725,COMPLETED,2024-02-15 20:30:51+00:00,2024-02-15 20:34:49+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:03:58,238.0,1
11,EXP_ZTASSETDISPOSE--GENERIC,3416056216,1075299782,COMPLETED,2024-02-15 20:15:09+00:00,2024-02-15 20:18:08+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:02:59,179.0,1
14,EXP_ZTASSETDISPOSE--GENERIC,3416054532,1075298940,COMPLETED,2024-02-15 20:01:06+00:00,2024-02-15 20:06:27+00:00,55175,-1,-1,-1,-1,-1,NaN,0,0 days 00:05:21,321.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,EXP_ZTASSETDISPOSE--GENERIC,3415399474,1074971411,COMPLETED,2024-02-12 01:02:17+00:00,2024-02-12 01:07:17+00:00,47193,-1,-1,-1,-1,-1,NaN,0,0 days 00:05:00,300.0,1
1114,EXP_ZTASSETDISPOSE--GENERIC,3415397430,1074970389,COMPLETED,2024-02-12 00:45:15+00:00,2024-02-12 00:48:46+00:00,47193,-1,-1,-1,-1,-1,NaN,0,0 days 00:03:31,211.0,1
1117,EXP_ZTASSETDISPOSE--GENERIC,3415395644,1074969496,COMPLETED,2024-02-12 00:30:23+00:00,2024-02-12 00:35:04+00:00,47193,-1,-1,-1,-1,-1,NaN,0,0 days 00:04:41,281.0,1
1120,EXP_ZTASSETDISPOSE--GENERIC,3415393966,1074968657,COMPLETED,2024-02-12 00:16:23+00:00,2024-02-12 00:20:05+00:00,47193,-1,-1,-1,-1,-1,NaN,0,0 days 00:03:42,222.0,1


In [55]:
df = job_runs[(job_runs['JOB_NAME'] == 'EXP_ZTASSETDISPOSE--GENERIC')][['ANOMALY'] + target_columns]


In [57]:
pd.get_option('display.max_rows')

60

In [60]:
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', 60)

In [63]:
df[df.ROWS_ADDED_COUNT == -1]

,ANOMALY,ROWS_ADDED_COUNT,ROWS_UPDATED_COUNT,ROWS_DELETED_COUNT,ROWS_DUPLICATED_COUNT,ROWS_SKIPPED_COUNT,ROWS_ERROR_COUNT
301,1,-1,-1,-1,-1,-1,-1
